In [29]:
import pandas as pd
import numpy as np

def conv_time_float(value):
    vals = value.split(':')
    t, hours = divmod(float(vals[0]), 24)
    t, minutes = divmod(float(vals[1]), 60)
    minutes = minutes / 60.0
    return hours + minutes


df = pd.read_csv("new2_input.csv")
df["Mealbreak"] = df["Mealbreak"].str[:-3]
df["Start"] = df["Start"].str[:-3]
df["End"] = df["End"].str[:-3]

df['pausa'] = df['Mealbreak'].apply(conv_time_float)
df['notturno'] = df['End'].apply(conv_time_float)
df['notturno'].mask(df['notturno'] > 7, 0, inplace=True)
df['totale'] = df['Total Hours']
df['data'] = df['Date']

df = df[['Employee Name','data','notturno','totale']]

from itertools import cycle

##repeat
df2 = df.loc[df.index.repeat(3)].reset_index(drop=True)

num_cycle = cycle([1, 2, 3])
df2['cat'] = [next(num_cycle) for num in range(len(df2))]

## repeat
df2 = df2.replace({'cat': {1: '_totale', 2: 'di cui notturni',3:'permessi'}})

df2['notturno'].mask(df2['cat'] == '_totale', 0, inplace=True)
df2['totale'].mask(df2['cat'] == 'di cui notturni', 0, inplace=True)
df2['notturno'].mask(df2['cat'] == 'permessi', 0, inplace=True)
df2['totale'].mask(df2['cat'] == 'permessi', 0, inplace=True)

df2['ore'] = df2['notturno']+df2['totale']
df2 = df2[['Employee Name','data','cat','ore']]
df2.to_excel('bau.xlsx',index=False)

In [43]:
df2 = pd.read_excel('bau.xlsx')
df2

,Employee Name,data,cat,ore
0,AMEDEO SANTOLINI,2022-08-01,_totale,6.5
1,AMEDEO SANTOLINI,2022-08-01,di cui notturni,2.5
2,AMEDEO SANTOLINI,2022-08-01,permessi,0.0
3,AMEDEO SANTOLINI,2022-08-03,_totale,7.5
4,AMEDEO SANTOLINI,2022-08-03,di cui notturni,2.5
...,...,...,...,...
1105,THOMAS VERSARI,2022-08-27,di cui notturni,0.0
1106,THOMAS VERSARI,2022-08-27,permessi,0.0
1107,THOMAS VERSARI,2022-08-31,_totale,7.0
1108,THOMAS VERSARI,2022-08-31,di cui notturni,0.0


In [44]:
df2 = df2.groupby(['Employee Name', 'data','cat']).sum()

In [46]:
df2

ore
Employee Name    data       cat                 
AMEDEO SANTOLINI 2022-08-01 _totale          6.5
                            di cui notturni  2.5
                            permessi         0.0
                 2022-08-03 _totale          7.5
                            di cui notturni  2.5
...                                          ...
THOMAS VERSARI   2022-08-27 di cui notturni  0.0
                            permessi         0.0
                 2022-08-31 _totale          7.0
                            di cui notturni  0.0
                            permessi         0.0

[996 rows x 1 columns]

In [48]:
df2 = df2.reset_index()
df2

,Employee Name,data,cat,ore
0,AMEDEO SANTOLINI,2022-08-01,_totale,6.5
1,AMEDEO SANTOLINI,2022-08-01,di cui notturni,2.5
2,AMEDEO SANTOLINI,2022-08-01,permessi,0.0
3,AMEDEO SANTOLINI,2022-08-03,_totale,7.5
4,AMEDEO SANTOLINI,2022-08-03,di cui notturni,2.5
...,...,...,...,...
991,THOMAS VERSARI,2022-08-27,di cui notturni,0.0
992,THOMAS VERSARI,2022-08-27,permessi,0.0
993,THOMAS VERSARI,2022-08-31,_totale,7.0
994,THOMAS VERSARI,2022-08-31,di cui notturni,0.0


In [50]:
multi = df2.set_index(['Employee Name', 'cat'])
multi['data'] = multi['data'].astype(str)
multi = multi.pivot(columns='data')
multi = multi.replace(np.nan,0)
multi

ore                        \
data                               2022-08-01 2022-08-02 2022-08-03   
Employee Name      cat                                                
AMEDEO SANTOLINI   _totale                6.5        0.0        7.5   
                   di cui notturni        2.5        0.0        2.5   
                   permessi               0.0        0.0        0.0   
CATERINA MILANDRI  _totale                6.0        5.5        0.0   
                   di cui notturni        2.0        1.5        0.0   
...                                       ...        ...        ...   
RICCARDO RICCIARDI di cui notturni        2.0        2.0        0.0   
                   permessi               0.0        0.0        0.0   
THOMAS VERSARI     _totale                0.0        6.5        6.5   
                   di cui notturni        0.0        0.0        0.0   
                   permessi               0.0        0.0        0.0   

                                                                     \
data                               2022-08-04 2022-08-05 2022-08-06   
Employee Name      cat                                                
AMEDEO SANTOLINI   _totale                9.0        0.0        7.0   
                   di cui notturni        2.0        0.0        3.0   
                   permessi               0.0        0.0        0.0   
CATERINA MILANDRI  _totale                0.0        0.0        0.0   
                   di cui notturni        0.0        0.0        0.0   
...                                       ...        ...        ...   
RICCARDO RICCIARDI di cui notturni        0.0        0.0        0.0   
                   permessi               0.0        0.0        0.0   
THOMAS VERSARI     _totale                0.0        7.0        7.5   
                   di cui notturni        0.0        0.0        0.5   
                   permessi               0.0        0.0        0.0   

                                                                     \
data                               2022-08-07 2022-08-08 2022-08-09   
Employee Name      cat                                                
AMEDEO SANTOLINI   _totale                8.5        8.5        8.0   
                   di cui notturni        2.5        2.0        2.0   
                   permessi               0.0        0.0        0.0   
CATERINA MILANDRI  _totale                0.0        0.0        0.0   
                   di cui notturni        0.0        0.0        0.0   
...                                       ...        ...        ...   
RICCARDO RICCIARDI di cui notturni        0.0        0.0        0.0   
                   permessi               0.0        0.0        0.0   
THOMAS VERSARI     _totale                7.5        0.0        0.0   
                   di cui notturni        0.5        0.0        0.0   
                   permessi               0.0        0.0        0.0   

                                               ...                        \
data                               2022-08-10  ... 2022-08-22 2022-08-23   
Employee Name      cat                         ...                         
AMEDEO SANTOLINI   _totale                5.0  ...        0.0        8.0   
                   di cui notturni        0.0  ...        0.0        1.5   
                   permessi               0.0  ...        0.0        0.0   
CATERINA MILANDRI  _totale                0.0  ...        0.0        8.5   
                   di cui notturni        0.0  ...        0.0        1.5   
...                                       ...  ...        ...        ...   
RICCARDO RICCIARDI di cui notturni        2.0  ...        0.0        1.0   
                   permessi               0.0  ...        0.0        0.0   
THOMAS VERSARI     _totale                7.0  ...        0.0        7.5   
                   di cui notturni        0.0  ...        0.0        0.0   
                   permessi               0.0  ...        0.0        